# Домашнее задание №2
В данном задании предлагается решить [контест](https://www.kaggle.com/competitions/vk-edu-bmstu-asr-competition/overview) на платформе Kaggle.
Основная цель данного задания состоит в том, чтобы студенты ознакомились с зоопарком открытых моделей для решения задачи распознавания речи, а так же попрактиковались в подходах по постпроцессингу распознанных гипотез для улучшения качества ASR.

Разрешается пользоваться всеми предобученными моделями.

**Строго запрещено размечать данные вручную!**

## Загрузка данных и инициализация модели

In [ ]:
#https://drive.google.com/file/d/1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG/view?usp=sharing # ссылка для ручной загрузки данных

In [1]:
!pip install gdown
!gdown 1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG && tar -xf kaggle.tar
import os
import soundfile as sf
import torch
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import string

Downloading...
From (original): https://drive.google.com/uc?id=1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG
From (redirected): https://drive.google.com/uc?id=1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG&confirm=t&uuid=6f205267-afb3-4c0e-a880-e22f343980c5
To: /kaggle/working/kaggle.tar
100%|████████████████████████████████████████| 138M/138M [00:01<00:00, 86.2MB/s]


Для базового решения задания воспользуемся предобученной моделью Whisper от OpenAI. Для этого обратимся к платформе [HuggingFace](https://huggingface.co/collections/openai/whisper-release-6501bba2cf999715fd953013)

## Baseline подход к инференсу модели

In [2]:
# загрузим модель
from transformers import WhisperProcessor, WhisperForConditionalGeneration

device = "cuda:0"
model_name = "openai/whisper-large"
processor = WhisperProcessor.from_pretrained(model_name)
processor.feature_extractor.return_attention_mask = True
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)



def batch_inference(model, processor, path_to_wavs, batch_size, sampling_rate=16000):
    results = {}
    wav_files = os.listdir(path_to_wavs)
    num_beams=2

    forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe") # generates task specific special tokens
    results = {}
    for i in tqdm(range(0, len(wav_files), batch_size), total=np.ceil(len(wav_files) / batch_size)):
        audio_paths = wav_files[i : i + batch_size]

        batch = []

        for path in audio_paths:
            audio, _ = sf.read(os.path.join(path_to_wavs, path))
            batch.append(audio)
        inputs = processor(batch, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

        x, x_masks = inputs["input_features"].to(device), inputs["attention_mask"].to(device)

        with torch.no_grad():
            model.eval()
            output_ids = model.generate(x, forced_decoder_ids=forced_decoder_ids, attention_mask=x_masks,num_beams = num_beams, num_return_sequences = num_beams, repetition_penalty=1.5, return_dict = True)

        transcribtion = processor.batch_decode(output_ids, skip_special_tokens=True)
        transcribtion = iter(transcribtion)
        
        for path in audio_paths:
            for j in range(num_beams):
                results.setdefault(path, []).append(next(transcribtion))


        
        
    return results

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

In [3]:
path_to_wavs = "kaggle/wav"
batch_size = 2
results = batch_inference(model=model, processor=processor, path_to_wavs=path_to_wavs, batch_size=batch_size)

100%|██████████| 321/321.0 [25:34<00:00,  4.78s/it]


In [5]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


def evaluate_confidence(text):
    inputs = tokenizer.encode_plus(text, 
                                    add_special_tokens=True, 
                                    max_length=512, 
                                    return_attention_mask=True, 
                                    return_tensors='pt')
    outputs = model(inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device))
    confidence = torch.nn.functional.softmax(outputs.last_hidden_state[:, 0, :], dim=1)
    return confidence.max().item()

# Пройдите по словарю и оставьте только одно предложение в каждом ключе
for key, values in results.items():
    confidences = [evaluate_confidence(value) for value in values]
    best_index = confidences.index(max(confidences))
    results[key] = [values[best_index]]


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=Tr

In [7]:
results

{'audio_464.wav': [' И в итоге...'],
 'audio_500.wav': [' Лисовская! Здравствуйте, дорогие девочки. Я обещаю...'],
 'audio_199.wav': [' и почти сразу сталкивается с первой серьезной опасностью. Княжеству грозит нападение шведов.'],
 'audio_136.wav': [' что?'],
 'audio_202.wav': [' Густья реки Невы в районе Ижоры.'],
 'audio_603.wav': [' Вот эта вот кнопка, она позволяет вам телепортироваться. Вы вот телепортируете вот сюда. Вы смотрите, я вам показываю указки, у вас есть...'],
 'audio_12.wav': [' Субтитры сделаны за поддержку Авиакатастроф.'],
 'audio_165.wav': [' Во-первых, хочется поздравить от всей души. Смешарики, с днем рождения. Лосяж! Не закругляйтесь!'],
 'audio_24.wav': [' Удивительный пилот'],
 'audio_122.wav': [' тем дешевле будет доставка. А если заказать самовывоз, экономия еще больше.'],
 'audio_477.wav': [' В общем, все более того, я могу сказать честно, я до конца, когда все это сложилось...'],
 'audio_402.wav': [' Итак, наши телемагистры примут участие в игре Тайного т

In [18]:
# torch.cuda.empty_cache()
# import gc
# gc.collect()

In [58]:
# Удаление "ТРЕВОЖНАЯ МУЗЫКА", "СМЕХ" и тп. 

#results = {key: '' if value.isupper() else value for key, value in results.items()}

In [9]:
# Преобразование "20" в "двадцать" и тп.

!pip install num2words
import num2words

# Функция для замены цифр на слова
def replace_numbers_with_words(text):
    import re
    numbers = re.findall(r'\d+', text)
    for number in numbers:
        text = text.replace(number, num2words.num2words(int(number), lang='ru'))
    return text

# Пройдите по словарю и замените цифры на слова
for key, values in results.items():
    new_values = []
    for value in values:
        new_value = replace_numbers_with_words(value)
        new_values.append(new_value)
    results[key] = new_values


In [47]:
# Удаление повторяющихся слов

# import re

# def remove_repeated_words(transcription):
#     transcription = re.sub(r'\b(\w+)\b\s+\1\b', r'\1', transcription)
#     return transcription
    
# results = {key: remove_repeated_words(value) for key, value in results.items()}

In [48]:
#Удаление неадекватно длинных слов

#results = {key: ' '.join([word for word in value.split() if len(word) <= 15]) for key, value in results.items()}

## Sample submission
Подготовим пример для загрузки на платформу

In [11]:
import string

def dummy_postprocessing(data):
    for filename, hypo in data.items():
        hypo = hypo[0].strip()  # берем первый элемент списка
        hypo = hypo.translate(str.maketrans('', '', string.punctuation))
        # hypo = hypo.replace('[', '').replace(']', '')  # удаляем скобки
        hypo = hypo.lower()
        if not hypo:
            data[filename] = "-"
        else:
            data[filename] = hypo  # сохраняем результат в виде списка
    return data

clean_data = dummy_postprocessing(results)


In [12]:
sample_submission = pd.read_csv("kaggle/sample_submission.csv")
sample_submission["hypo"] = sample_submission["filename"].apply(lambda x: clean_data[x])
sample_submission.insert(0, "id", sample_submission.index)
sample_submission.to_csv("baseline.csv", index=False)

In [13]:
sample_submission.head(10)

,id,filename,hypo
0,0,audio_0.wav,тревожная музыка
1,1,audio_1.wav,хахаха
2,2,audio_2.wav,тише жрём враг на гремле сюда идти я сказал
3,3,audio_3.wav,слышали не понял
4,4,audio_4.wav,быстро за мной
5,5,audio_5.wav,ты где весь сентябрь уроки прогулил вообще
6,6,audio_6.wav,наркоманец телевизор твой из дома вынесет ну м...
7,7,audio_7.wav,я в топтридцать по батлтимс два
8,8,audio_8.wav,у меня реакция сто сорок миллисекунд витя толь...
9,9,audio_9.wav,значит так глеб с завтрашнего дня берешься за ...


In [34]:
pwd

'/kaggle/working'

In [14]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink('baseline.csv')

/kaggle/working


/kaggle/working/baseline.csv

## Идеи по улучшению предсказаний:
* Всегда ли LAS архитектура работает на тишине корректно?
* Как насчет нормализации текста? (19.02 - девятнадцатое февраля, 20 лет - двадцать лет)
* Может быть попробовать рескоринг? (возвращаем больше, чем топ-1 гипотезу из бима)
* Может быть взять модель побольше?
## Соревнование доступно по [ссылке](https://www.kaggle.com/t/325b553468db4d01a583e29624a7685d)